In [43]:
import numpy as np
import ase
from  ase.io import read,write
from ase.calculators.emt import EMT
from ase.build import bulk
from ase.io import Trajectory
from ase.visualize import view

In [44]:
# First, we make a good initial guess for a using FCC nearest neighbor distance
a0 = 3.52 / np.sqrt(2)
c0 = np.sqrt(8/3.0) * a0

In [45]:
traj = Trajectory("Ni.traj", 'w')

In [46]:
ni = bulk("Ni", 'hcp', a=a0, c=c0)
# view(ni)
ni.cell

Cell([[2.4890158697766473, 0.0, 0.0], [-1.2445079348883237, 2.1555509736491967, 0.0], [0.0, 0.0, 4.064545895094965]])

In [47]:
eps = 0.01
for a in a0*np.linspace(1-eps, 1+eps, 3):
    for c in c0*np.linspace(1-eps, 1+eps, 3):
        ni = bulk("Ni", 'hcp', a,c)
        ni.calc = EMT()
        ni.get_potential_energies()
        traj.write(ni)

In [48]:
configs = read('Ni.traj@:')
energies = [config.get_potential_energy() for config in configs]
a = np.array([config.cell[0,0] for config in configs])
c = np.array([config.cell[2,2] for config in configs])
a, energies

(array([2.46412571, 2.46412571, 2.46412571, 2.48901587, 2.48901587,
        2.48901587, 2.51390603, 2.51390603, 2.51390603]),
 [-0.029701330447096552,
  -0.027509801944638923,
  -0.020589818669460414,
  -0.024753432930204156,
  -0.020417223624274783,
  -0.011530632137137786,
  -0.008699207576530199,
  -0.002135271279630757,
  0.008801194926746803])

In [50]:
# energy = p0 + p1*a + p2*c + p3*a**2 + p4*a*c + p5*c**2
functions = np.array([a**0, a, c, a**2, a * c, c**2])
p = np.linalg.lstsq(functions.T, energies, rcond=-1)[0]
p

array([ 97.72461862, -52.8738722 , -16.19193701,   9.0307395 ,
         2.07302647,   1.37721208])

In [63]:
# 以下操作即求解 energy = p0 + p1*a + p2*c + p3*a**2 + p4*a*c + p5*c**2
# 的a或者c的偏导=0，即能量最小点取得a或者c
p0 = p[0]
p1 = p[1:3]
p2 = np.array([(2 * p[3], p[4]),
               (p[4], 2 * p[5])])
print(p0)
print(p1)
p2

97.72461862259729
[-52.8738722  -16.19193701]


array([[18.06147899,  2.07302647],
       [ 2.07302647,  2.75442416]])

In [62]:

a0, c0 = np.linalg.solve(p2.T, -p1)
a0,c0

(2.4657199396330904, 4.022777044194669)